In [1]:
!pip install retrying
!pip install fake_useragent
from bs4 import BeautifulSoup
from retrying import retry
import requests
import pandas as pd
import sys
import time
import random
from urllib.parse import quote
from fake_useragent import UserAgent


In [9]:
URL = "https://www.amazon.com/Apple-iPhone-11-64GB-Black/dp/B07ZPKN6YR/ref=sr_1_1?crid=39DAZO9YLWW6V&keywords=iphone&qid=1700427702&sprefix=iphone%2Caps%2C331&sr=8-1&th=1"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US, en;q=0.5'
}
# Créez une instance de UserAgent
ua = UserAgent()

In [11]:
webpage = requests.get(URL, headers=HEADERS)
print(webpage.status_code)

200


In [12]:
soup = BeautifulSoup(webpage.content, "html.parser")
reviews = []
link = []
for b in soup.findAll("a", {'data-hook': "see-all-reviews-link-foot"}):
  link.append(b['href'])
print("links", link)

links ['/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR?reviewerType=all_reviews', '/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR?reviewerType=all_reviews']


In [14]:
# Function to make a request using Crawlbase
CRAWLBASE_TOKEN = '6CtC079r3M21jspX_DPN2g'  # Your Crawlbase token
def make_request(url):
    encoded_url = quote(url, safe='')
    crawlbase_url = f'https://api.crawlbase.com/?token={CRAWLBASE_TOKEN}&url={encoded_url}'
    response = requests.get(crawlbase_url, headers={'User-Agent': ua.random})
    return response

In [19]:
from concurrent.futures import ThreadPoolExecutor

# Your other functions and imports...

MAX_RETRIES = 2
RETRY_SLEEP_SECONDS = 2

def make_request_with_retry(url):
    for attempt in range(MAX_RETRIES):
        page = make_request(url)

        if page.status_code == 200:
            return page
        else:
            print(f"Error fetching page {url} - Attempt {attempt + 1} - Status Code: {page.status_code}")
            #print("Response Body:", page.text)
            time.sleep(RETRY_SLEEP_SECONDS)

    print(f"Failed to fetch page {url} after {MAX_RETRIES} attempts.")
    return None

def process_page(url):
    page = make_request_with_retry(url)

    if page is not None and page.status_code == 200:
        print("Page", url, "fetched successfully")
        soup = BeautifulSoup(page.content, "html.parser")

        new_reviews = [i.text for i in soup.findAll("span", {'data-hook': "review-body"})]
        if not new_reviews:
            print("No new reviews found on page", url)
            return None

        time.sleep(random.uniform(2, 4))
        return new_reviews
    else:
        return None

if link:
    new_link = "https://www.amazon.com" + link[0]
    urls = [new_link + f'&pageNumber={k}' for k in range(1, 13)]  # Adjust the range as needed

    with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust the number of workers as needed
        reviews = list(filter(None, executor.map(process_page, urls)))

    if reviews:
        reviews = [review for sublist in reviews for review in sublist]
        rev = {'reviews': reviews}
        review_data = pd.DataFrame.from_dict(rev)
        review_data.to_csv("amazon_data.csv", header=True, index=False)
        print("Total reviews collected:", len(reviews))
    else:
        print("No reviews collected.")
else:
    print("Error fetching initial page - Status Code:", webpage.status_code)

Page https://www.amazon.com/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR?reviewerType=all_reviews&pageNumber=2 fetched successfully
Page https://www.amazon.com/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR?reviewerType=all_reviews&pageNumber=5 fetched successfully
Page https://www.amazon.com/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR?reviewerType=all_reviews&pageNumber=6 fetched successfully
Page https://www.amazon.com/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR?reviewerType=all_reviews&pageNumber=7 fetched successfully
Page https://www.amazon.com/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR?reviewerType=all_reviews&pageNumber=8 fetched successfully
Page https://www.amazon.com/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR?reviewerType=all_reviews&pageNumber=9 fetched successfully
Page https://www.amazon.com/Apple-iPhone-11-64GB-Black/product-reviews/B07ZPKN6YR?reviewerType=all_reviews&pageNumber=3 fetched successfully
Page https://

In [20]:
print(review_data)

                                              reviews
0   \nMy iPhone 6 died and was only iOS 10 so it w...
1   \nThe renewed iPhone 11 itself is in great con...
2   \nI'll be honest with you, at first I was a bi...
3   \nI recently purchased the renewed Apple iPhon...
4   \nI purchased this iPhone 11 and paid extra fo...
..                                                ...
95  \nthis is probably one of the most useful purc...
96  \ngreat phone and half the price. just put my ...
97  \nThe phone is perfect my son loves it! 🥰 it c...
98  \nIt works so good already I just got it I’m s...
99  \nThis is a great phone. Refurbished is the wa...

[100 rows x 1 columns]


In [23]:
# Afficher un review en entier
if not review_data.empty:
    review = review_data['reviews'].iloc[2]
    print("Full text of the first review:\n", review)
else:
    print("Aucune revue n'a été collectée.")

Full text of the first review:
 
I'll be honest with you, at first I was a bit skeptical. I was told to avoid "Renewed" phones because unlike Apple-refurbished phones, different refurbishment factories are sourced online, and the fact there are only stock images rather than actual pictures of the phone someone took. My first concern was from the other reviews I've read that range from 70% battery capacity, to detecting random touch screen inputs, to speakers not working, to vibration modules not working at all, which seemed like buying refurb phones from the Internet is a pretty big gamble. Fortunately, and thankfully, I have found a diamond in the rough. This was a refurb I bought from Stone Digital. This 11 didn't bug me about "non-genuine Apple parts" messages, didn't have touch screen problems (at least I put the screen protector on before turning it on for the first time if that helps), Speakers (both external and head speaker) worked fine, SIM card reader worked fine, Battery Cap